# API de dados agregados do IBGE
https://servicodados.ibge.gov.br/api/docs/agregados?versao=3

## configurações

In [ ]:
import urllib.request
import json
import pandas as pd
import numpy as np
import datetime
import matplotlib.pyplot as plt
import seaborn as sns

### funções personalizadas

In [ ]:
def IBGE_tratar_listas(df, coluna):
    """
    dados que existem em listas dentro de uma coluna.
    # utilizar o explode
    # ideia em :https://stackoverflow.com/questions/27263805/pandas-column-of-lists-create-a-row-for-each-list-element
    """
    # verificar se é uma lista
    if isinstance(df[coluna][0], list):
        pass
    else:
        print(f"a função IBGE_tratar_listas precisa receber uma coluna com conteúdo do tipo list.\
        \nA coluna passada possui o tipo {type(df[coluna][0])}.\
        \nRetornando dataframe original.")
        return df

    df = df.explode(coluna).reset_index(drop=True)
    return df

def IBGE_tratar_json(df, coluna):
    """
    dados em json dentro de uma coluna.
    """
    df = pd.concat([df, df[coluna].apply(pd.Series)], axis=1)
    return df

In [ ]:
# query builder
# to do: criar função para automatizar o builder

# Índice Nacional de Preços ao Consumidor Amplo
## 7060: IPCA - Variação mensal, acumulada no ano, acumulada em 12 meses e peso mensal, para o índice geral, grupos, subgrupos, itens e subitens de produtos e serviços (a partir de janeiro/2020)
## 1737: IPCA - Série histórica com número-índice, variação mensal e variações acumuladas em 3 meses, em 6 meses, no ano e em 12 meses (a partir de dezembro/1979)
# Produção da Extração vegetal e da silvicultura
## 5930- Área total existente em 31/12 dos efetivos da silvicultura, por espécie florestal
## 291 - - Quantidade produzida e valor da produção na silvicultura, por tipo de produto da silvicultura

var_agregado = 7060

# PERÍODO
# Pode conter um ou mais períodos delimitados pelo caracter | (pipe)
# pode conter um intervalo segundo o padrão <INICIO>-<FIM>
# valores negativos -6: Obtém os resultados nos últimos seis períodos pesquisados (-6)
# var_periodos = -12
# multiplos períodos podem ser selecionados com o caracter pipe (|)
var_periodos ='202001|202002|202003|202004|202005|202006|\
202007|202008|202009|202010|202011|202012|\
202101|202102|202103|202104|202105|202106|\
202107|202108|202109|202110|202111|202112|\
202201|202202|202203|202204|202205|202206|\
202207|202208|202209|202210|202211'

# VARIÁVEIS
var_variáveis = '63|69|2265'

# LOCALIDADES

# CLASSIFICAÇÕES
var_localidades = 'N1[all]'
var_classificação = '315[all]'

API_IBGE = 'https://servicodados.ibge.gov.br/api/v3/'
agregado = 'agregados/'+str(var_agregado)+'/'
período = 'periodos/'+str(var_periodos)+'/'
variáveis = 'variaveis/'+str(var_variáveis)+'?'
localidades = 'localidades='+str(var_localidades)
classificações = '&classificacao='+str(var_classificação)


query = API_IBGE+agregado+período+variáveis+localidades+classificações
query_test = 'https://servicodados.ibge.gov.br/api/v3/agregados/7060/periodos/202001|202002|202003|202004|202005|202006|202007|202008|202009|202010|202011|202012|202101|202102|202103|202104|202105|202106|202107|202108|202109|202110|202111|202112|202201|202202|202203|202204|202205|202206|202207|202208|202209|202210|202211/variaveis/63|69|2265?localidades=N1[all]&classificacao=315[all]'


# Área total existente em 31/12 dos efetivos da silvicultura
#query = 'https://servicodados.ibge.gov.br/api/v3/agregados/5930/periodos/2013%7C2014%7C2015%7C2016%7C2017%7C2018%7C2019%7C2020%7C2021/variaveis/6549?localidades=N3[all]&classificacao=734[39326,39327,39328]'

# IPCA - Variação mensal
#query = 'https://servicodados.ibge.gov.br/api/v3/agregados/1737/periodos/197912|198001|198002|198003|198004|198005|198006|198007|198008|198009|198010|198011|198012|198101|198102|198103|198104|198105|198106|198107|198108|198109|198110|198111|198112|198201|198202|198203|198204|198205|198206|198207|198208|198209|198210|198211|198212|198301|198302|198303|198304|198305|198306|198307|198308|198309|198310|198311|198312|198401|198402|198403|198404|198405|198406|198407|198408|198409|198410|198411|198412|198501|198502|198503|198504|198505|198506|198507|198508|198509|198510|198511|198512|198601|198602|198603|198604|198605|198606|198607|198608|198609|198610|198611|198612|198701|198702|198703|198704|198705|198706|198707|198708|198709|198710|198711|198712|198801|198802|198803|198804|198805|198806|198807|198808|198809|198810|198811|198812|198901|198902|198903|198904|198905|198906|198907|198908|198909|198910|198911|198912|199001|199002|199003|199004|199005|199006|199007|199008|199009|199010|199011|199012|199101|199102|199103|199104|199105|199106|199107|199108|199109|199110|199111|199112|199201|199202|199203|199204|199205|199206|199207|199208|199209|199210|199211|199212|199301|199302|199303|199304|199305|199306|199307|199308|199309|199310|199311|199312|199401|199402|199403|199404|199405|199406|199407|199408|199409|199410|199411|199412|199501|199502|199503|199504|199505|199506|199507|199508|199509|199510|199511|199512|199601|199602|199603|199604|199605|199606|199607|199608|199609|199610|199611|199612|199701|199702|199703|199704|199705|199706|199707|199708|199709|199710|199711|199712|199801|199802|199803|199804|199805|199806|199807|199808|199809|199810|199811|199812|199901|199902|199903|199904|199905|199906|199907|199908|199909|199910|199911|199912|200001|200002|200003|200004|200005|200006|200007|200008|200009|200010|200011|200012|200101|200102|200103|200104|200105|200106|200107|200108|200109|200110|200111|200112|200201|200202|200203|200204|200205|200206|200207|200208|200209|200210|200211|200212|200301|200302|200303|200304|200305|200306|200307|200308|200309|200310|200311|200312|200401|200402|200403|200404|200405|200406|200407|200408|200409|200410|200411|200412|200501|200502|200503|200504|200505|200506|200507|200508|200509|200510|200511|200512|200601|200602|200603|200604|200605|200606|200607|200608|200609|200610|200611|200612|200701|200702|200703|200704|200705|200706|200707|200708|200709|200710|200711|200712|200801|200802|200803|200804|200805|200806|200807|200808|200809|200810|200811|200812|200901|200902|200903|200904|200905|200906|200907|200908|200909|200910|200911|200912|201001|201002|201003|201004|201005|201006|201007|201008|201009|201010|201011|201012|201101|201102|201103|201104|201105|201106|201107|201108|201109|201110|201111|201112|201201|201202|201203|201204|201205|201206|201207|201208|201209|201210|201211|201212|201301|201302|201303|201304|201305|201306|201307|201308|201309|201310|201311|201312|201401|201402|201403|201404|201405|201406|201407|201408|201409|201410|201411|201412|201501|201502|201503|201504|201505|201506|201507|201508|201509|201510|201511|201512|201601|201602|201603|201604|201605|201606|201607|201608|201609|201610|201611|201612|201701|201702|201703|201704|201705|201706|201707|201708|201709|201710|201711|201712|201801|201802|201803|201804|201805|201806|201807|201808|201809|201810|201811|201812|201901|201902|201903|201904|201905|201906|201907|201908|201909|201910|201911|201912|202001|202002|202003|202004|202005|202006|202007|202008|202009|202010|202011|202012|202101|202102|202103|202104|202105|202106|202107|202108|202109|202110|202111|202112|202201|202202|202203|202204|202205|202206|202207|202208|202209|202210|202211/variaveis/63|2263|2264|69|2265?localidades=N1[all]'

In [ ]:
print(query)
print(query_test)
print(f"queries iguais= {query==query_test}")

In [ ]:
 
with urllib.request.urlopen(query) as url:
    data = url.read().decode('utf-8')
    data = json.loads(data)

***

https://www.skytowner.com/explore/splitting_dictionary_into_separate_columns_in_pandas_dataframe

In [ ]:
df = pd.read_json(json.dumps(data))

In [ ]:
df

## Data Clean

In [ ]:
df_clean = df.copy()

In [ ]:
# remover id para evitar conflitos, pois existem diversos ids na base
df_clean.drop(columns='id', inplace=True)


### coluna resultados

In [ ]:
# a coluna 'resultados' contém uma lista para cada UF,grupo, subgrupo, etc.
df_clean = IBGE_tratar_listas(df_clean, 'resultados')

# a coluna 'resultados' é expandidada em 'classificacoes' e 'series'
df_clean = IBGE_tratar_json(df_clean, 'resultados')

# remover coluna
df_clean.drop(columns='resultados',inplace=True)

### coluna classificações

In [ ]:
df_clean = IBGE_tratar_listas(df_clean, 'classificacoes')
df_clean = IBGE_tratar_json(df_clean, 'classificacoes')

# remover coluna
df_clean.drop(columns='classificacoes',inplace=True)

In [ ]:
# COL CATEGORIA
# 2. na coluna 'categoria' existem os ids e nome das categorias em um dict. Manter apenas as categorias.
# Ideia para extrair valores de um dict em ### https://bobbyhadz.com/blog/python-get-first-value-in-dictionary
df_clean['categoria'] = df_clean['categoria'].apply(lambda x: list(x.values())[0])

In [ ]:
df_clean.head()

### coluna 'series'

In [ ]:
# a coluna 'series' contém uma lista para cada localidade.

df_clean = IBGE_tratar_listas(df_clean, 'series')
df_clean = IBGE_tratar_json(df_clean, 'series')

# remover coluna
df_clean.drop(columns='series', inplace=True)

In [ ]:
df_clean.head()

In [ ]:
# expandir 'localidade'

df_clean = IBGE_tratar_listas(df_clean, 'localidade')
df_clean = IBGE_tratar_json(df_clean, 'localidade')

# remover coluna
df_clean.drop(columns='localidade', inplace=True)

In [ ]:
df_clean.head()

In [ ]:
# coluna 'series' dividida em 'localidade' e 'serie'
df_clean = pd.concat([df_clean, df_clean['localidade'].apply(pd.Series)], axis=1)
df_clean = pd.concat([df_clean, df_clean['serie'].apply(pd.Series)], axis=1)
df_clean.drop(columns=['localidade','serie'], inplace=True)

In [ ]:
df_clean

In [ ]:
# localidade dividida em 'id', 'nivel' e 'nome'
# remover 'id' e 'nivel'
df_clean.drop(columns=['id','nivel'], inplace=True)

In [ ]:
df_clean

In [ ]:
df_clean.variavel[0]

In [ ]:
df_clean = df_clean.melt(id_vars=['variavel','unidade','nome'], var_name='ano/mês',value_name='valor')

### formatar datas

In [ ]:
def format_anomes(data_str):
    """"""
    ano = int(data_str[:4])
    mês = int(data_str[-2:])
    dt = datetime.date(ano,mês, 1)
    return dt.strftime('%b/%Y')

df_clean['ano/mês'] = df_clean['ano/mês'].apply(lambda x: format_anomes(x))

### itens nulos e formatos 

In [ ]:
df_clean.info()

### corrigir NANs

In [ ]:
df_clean.replace(to_replace=['-','...'], value=np.nan, inplace=True)

In [ ]:
# corrigir formatos
df_clean['valor'] = df_clean['valor'].astype('float')

In [ ]:
df_clean

In [ ]:
nome_arquivo = 'IPCA_série_histórica.csv'

df_clean.to_csv(nome_arquivo, encoding='LATIN-1', index=False, sep=';', decimal=',')

***

## data analysis

In [ ]:
df_plot = df_clean[['espécie','ano','hectares']].groupby(['espécie','ano']).sum().reset_index()
df_plot

In [ ]:
plt.scatter(df_plot['ano'], df_plot['hectares'])

In [ ]:
sns.scatterplot(data=df_clean, x='ano', y='hectares', style='espécie', hue='espécie', size='hectares')